#TensorFlow Tutorial

본 문서는 TensorFlow 를 사용하여 기초적인 Deep Learning을 구현하고 실험하기 위한 실습 자료이다. TensorFlow에 대한 기본 원리 및 deep learning 구현 예제를 다루어보고, TensorFlow로 구현 후 공개된 여러 오픈소스를 둘러본다.

The code and comments are written by Dong-Hyun Kwak (imcomking@gmail.com)

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.


## TensorFlow

TensorFlow 이하, tf는 구글 주도하에 초기 개발되어, 2015년 12월 부터 오픈소스로 공개되어 널리 쓰이고 있는 기계학습(딥러닝)을 위한 라이브러리이다. Computational Graph를 사용한 Theano의 장점을 그대로 살려 automatic gradient 계산이 가능하고, Multi-GPU 환경에서도 동작가능한 아키텍처 기반으로 설계되었다.

TensorFlow는 CPU와 GPU, Multi-GPU 환경을 모두 지원하며, 함수의 적절한 추상화 수준, 직관적이고 쉬운 문법, 빠른 업데이트 속도, 구글의 강력한 지원, 넓은 사용자 층 등이 TensorFlow가 가진 큰 장점이다.


#### 관련 사이트
TensorFlow 공식 홈페이지 : http://tensorflow.org/
<br>TensorFlow Github : https://github.com/tensorflow/tensorflow/releases
<br>TensorFlow Contributors : https://github.com/tensorflow/tensorflow/graphs/contributors
<br>TensorFlow Playground : http://playground.tensorflow.org/
<br>딥러닝 Docker Image : https://hub.docker.com/r/imcomking/bi_deeplearning/
<br>강의자료 : https://github.com/bi-lab/deeplearning_tutorial

##TensorFlow License : Apache 2.0

<img src="images/tf_license.png" width=600 alt="hi" class="inline"/>


https://github.com/tensorflow/tensorflow/blob/master/LICENSE








### Apache 2.0 License :
자유로운 상업적 이용, 수정, 파생물 창작 및 배포가 가능하며 이 소프트웨어를 이용한 창작물을 공개하지 않아도 된다.
단, TensorFlow 로고는 따로 사용할 수 없고 반드시 저작권, 라이선스, 공지사항, 변경사항 문서를 첨부해야한다.

![](images/apache2.0_license.png)
https://tldrlegal.com/license/apache-license-2.0-%28apache-2.0%29

### TensorFlow의 핵심 원리
Computation Graph와 Session은 tf를 이해하는 데 가장 중요한 핵심 개념이다. 이 두가지 요소만 이해하면 전체적인 원리와 흐름을 모두 파악할 수 있다.

Computation Graph란 tf에서 실행되는 모든 알고리즘 혹은 모델을 나타내는 것이라 말할 수 있다. 보다 쉽게 설명하자면, tf에서는 하나의 거대한 함수가 실행되는데 이 함수는 내부적으로 수학적인 여러 계산을 통해 구성되어 있다. 이러한 계산 과정은 일련의 node와 edge들의 연결로써 하나의 graph를 이루게 되는데 이것이 바로 Computation Graph이다. 

그래서 성공적으로 모델을 작성하게 되면 아래와 같은 Computation Graph(MLP의 feedforward 과정)가 만들어 진다. Computation Graph 기반의 프레임워크가 가진 장점은 모델 구현의 유연성이 크고, 자동화된 미분 계산이 가능하다는 점이 있다.


Session이란 한마디로 Computation Graph가 실행되는 환경 혹은 자원을 의미한다. 일반적으로 딥러닝 연산에는 GPU를 활용하게 되는데, Session은 이러한 GPU장치에서 확보한 메모리 및 프로세서 자원을 추상화시켜 표현한 것이다.

즉 이 두가지를 직관적으로 비유하자면, Computation Graph는 딥러닝 모델이고, Session은 이 모델이 실행되는 GPU라고 할 수 있다.

<img src = "images\tensorboard_mlp.png" width=800>

그러면 우선 기본적인 tf의 문법과 형식을 익히기 위해, 가장 기본적인 Deep Learning인 MLP의 Computation Graph를 아무것도 없는 상태에서 만들어보는 실습을 해보자.


### Multi-Layer Perceptron
Multi-Layer Perceptron, MLP는 다음과 같은 구조를 가진 모델이다. Convolutional Neural Networks와 달리 굉장히 layer간의 연결이 빽빽하게 가득 차 있어, dense layer 혹은 fully connected layer라는 이름으로도 불린다.

<img src = "images\mlp.png" width=400>
(출처: http://blog.refu.co/?p=931)

그러면 이러한 MLP를 이용해서 MNIST 데이터를 분류하는 코드를 작성해보자. MNIST 데이터는 다음과 같은 사람이 쓴 숫자 손글씨 7만장을 모아놓은 것으로, 기계학습에서 아주 널리 사용되는 데이터셋이다.

MNIST 데이터는 아래와 같이 28x28 크기의 이미지에 gray scale로 숫자가 그려져있고, 각 이미지가 어떤 숫자인지 label 정보가 달려있다.

<img src = 'images\mnist.png' width=450>

그러면 가장 먼저 이를 구현하기위해서는 사용할 라이브러리를 import 해주어야 한다.

In [31]:
%matplotlib inline
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
print (tf.__version__)

1.0.1


그리고 실습에서 사용할 MNIST 데이터를 다음과 같이 다운로드 받는다.

In [32]:
# download the mnist data.
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


이제 mnist 데이터를 저장시킬 x와 y_target 변수를 선언해야한다.
tf.placeholder는 우리가 원하는 데이터를 Computation Graph에 입력해주는 역할을 하는 변수이다. 즉 input data를 받기 위한 변수라고 생각할 수 있다.

In [33]:
# placeholder is used for feeding data.
x = tf.placeholder("float", shape=[None, 784]) # none represents variable length of dimension. 784 is the dimension of MNIST data.
y_target = tf.placeholder("float", shape=[None, 10]) # shape argument is optional, but this is useful to debug.

데이터를 넣을 변수를 생성했으니, 이제 실제 MLP에서 사용할 변수들을 생성하고, 이들을 이용해 Computation Graph를 그려본다.

In [34]:
# all the variables are allocated in GPU memory
W1 = tf.Variable(tf.zeros([784, 256]))      # create (784 * 256) matrix
b1 = tf.Variable(tf.zeros([256]))           # create (1 * 256) vector
weighted_summation1 = tf.matmul(x, W1) + b1 # compute --> weighted summation
h1 = tf.sigmoid( weighted_summation1 )      # compute --> sigmoid(weighted summation)

# Repeat again
W2 = tf.Variable(tf.zeros([256, 10]))        # create (256 * 10) matrix
b2 = tf.Variable(tf.zeros([10]))             # create (1 * 10) vector
weighted_summation2 = tf.matmul(h1, W2) + b2 # compute --> weighted summation
y = tf.nn.softmax(weighted_summation2)       # compute classification --> softmax(weighted summation)

위의 과정까지 완료된 경우, 변수 y는 3층짜리 MLP에서 input data에 대해 label을 예측한 결과가 저장된다. 지금까지 구현한 것은 MLP의 feed-forward process에 해당하는 Computation Graph이다.

그러면 이제 MLP를 학습시키기 위한, 수학적인 연산들을 정의해보자.

In [35]:
# define the Loss function
cross_entropy = -tf.reduce_sum(y_target*tf.log(y))

cross entorpy는 Deep Learning의 Classificatoin 모델에서 일반적으로 사용하는 에러함수이다. 간단히 말해 이 cross entropy는 MLP가 예측한 y 값이 실제 데이터와 다른 정도를 확률적으로 측정한다고 볼 수 있다.

MLP가 학습 되기위해서는 이 에러함수가 최대한 작은 값을 내도록 만들어야한다. 그래서 이 에러함수를 각각의 변수들로 편미분하여 gradient를 계산하고, 에러가 최소가 되는 변수값을 찾아가는 것이 바로 MLP의 학습 알고리즘이다.

이를 구현하려면 원래 미분 후 에러가 줄어드는 방향으로 변수를 이동시키는 과정등을 직접 코딩해야하지만, 자동 미분을 제공하는 tf에서는 아래의 단 한줄로 구현할 수 있다.

In [36]:
# define optimization algorithm
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

이제 학습 알고리즘의 구현이 끝났지만, 아직 더 필요한 기능이 있다. MLP가 잘 학습하고 있는지 성능을 모니터링하기 위한 정확도 계산을 정의해보자. 이 기능이 없으면 학습이 잘되고있는지를 전혀 파악할 수 없어 overfitting이 발생해도 해결할 수가 없다.

In [37]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))
# correct_prediction is list of boolean which is the result of comparing(model prediction , data)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
# tf.cast() : changes true -> 1 / false -> 0
# tf.reduce_mean() : calculate the mean

correct_prediction 는 뭔가 수식이 복잡해 보이지만 사실 매우 간단한 계산이다. 먼저 tf.argmax(y, 1) 함수는 y 벡터 중에서 가장 값이 큰 index를 알려주는 함수이다. 즉 모델이 예측한 class를 나타낸다. 그래서 모델이 예측한 class와 실제 데이터에 labeling된 class를 비교하여 같으면 true, 다르면 false를 내도록 계산한 것이 바로 correct_prediction이다.

accuracy는 앞서 계산한 correct_prediction 이라는 true/false 리스트를 1과 0으로 변환한 뒤, 이를 평균낸 것이다.

그러면 이제 필요한 모든 Computation Graph를 정의하였으니 이제 이를 session을 이용하여 실행만 시키면 된다.
일반적으로 하나의 Computation Graph는 하나의 session에 의해서 실행된다. 그럼 session을 생성해보자.

In [38]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True))) # open a session which is a envrionment of computation graph.
sess.run(tf.global_variables_initializer())# initialize the variables

gpu_options=tf.GPUOptions(allow_growth =True)) 옵션은 session이 필요한 최소한의 자원만 할당해서 사용할 것을 강제하는 옵션이다.

tf.initialize_all_variables() 는 앞서 생성했던 변수들을 사용하기 위해서 반드시 실행해야하는 초기화 단계이다. 이 연산은 gpu 메모리에 실제로 값을 할당하는 기능을 하기 때문에, 반드시 session에 의해서 실행되어야한다. session으로 실행을 하기위해서는 위와 같이 sess.run() 함수를 이용해 필요한 연산을 실행시킨다.

우선은 Ctrl + Enter를 눌러 다음 코드를 실행 시켜보자.

In [39]:
# training the MLP
for i in range(5001): # minibatch iteraction
    batch = mnist.train.next_batch(100) # minibatch size
    sess.run(train_step, feed_dict={x: batch[0], y_target: batch[1]}) # feed data into placeholder x, y_target

    if i%500 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_target: batch[1]})
        print ("step %d, training accuracy: %.3f"%(i, train_accuracy))

# for given x, y_target data set
print  ("validation accuracy: %g"% sess.run(accuracy, feed_dict={x: mnist.validation.images, y_target: mnist.validation.labels}))
sess.close()
tf.reset_default_graph()

step 0, training accuracy: 0.120
validation accuracy: 0.0958
step 500, training accuracy: 0.740
validation accuracy: 0.6214
step 1000, training accuracy: 0.830
validation accuracy: 0.758
step 1500, training accuracy: 0.910
validation accuracy: 0.8744
step 2000, training accuracy: 0.940
validation accuracy: 0.906
step 2500, training accuracy: 0.970
validation accuracy: 0.9116
step 3000, training accuracy: 0.930
validation accuracy: 0.9124
step 3500, training accuracy: 0.970
validation accuracy: 0.9094
step 4000, training accuracy: 0.960
validation accuracy: 0.9168
step 4500, training accuracy: 0.950
validation accuracy: 0.9206
step 5000, training accuracy: 0.970
validation accuracy: 0.9258


총 5001번의 minibatch iteraction을 실행하고, 500번 마다 학습 정확도를 측정, 그리고 마지막에는 validation 정확도를 측정하고 있다.

코드는 간단한 구조이다. for 문이 전체 iteration을 실행하고, 맨 처음 가져왔던 mnist 데이터를 100개씩 가져와서 placeholder에 넣어준다. 그리고 sess.run()을 통해 위에서 정의했던 학습 알고리즘과 정확도 계산을 실행한다.

이제 우리가 만든 MLP의 구조를 직접 눈으로 확인해보자.

### TensorBoard 설정하기
TensorFlow는 TensorBoard라는 매우 강력한 visualization tool을 제공한다. 이를 사용하면 웹브라우저에서 사용자가 모델의 구조를 눈으로 확인하고, 파라미터 값의 분포를 살펴보는 등의 직관적인 분석이 가능하다. 다만, 아직은 실시간 데이터 업데이트 기능이 구현되지 않아 학습과정을 2분에 한번만 모니터링할 수가 있다. TensorBoard에는 현재도 수많은 기능이 구현중에 있고, 여러 오픈소스 개발자들의 기여를 기다리고 있다. (https://github.com/tensorflow/tensorflow/issues/2603)

그러면 이 TensorBoard를 활용해 방금 만들었던 MLP를 분석해보자. 그러려면 다음의 사항을 반영해 코드를 수정하여야 한다.

* **변수들의 이름 지어주기**

* **변수들의 Summary 생성**

* **변수들의 Summary 기록**

아래의 코드는 위의 3가지 사항을 모두 반영하고, MLP 코드를 하나의 파이썬 함수로 정리한 코드이다. 세세한 차이는 위에서 우리가 짰던 코드와 비교를 하면 파악이 가능하다.

In [40]:
%matplotlib inline
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.reset_default_graph() # remove the previous computation graph

def MLP():
    # download the mnist data.
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True) 


    # placeholder is used for feeding data.
    x = tf.placeholder("float", shape=[None, 784], name = 'x') # none represents variable length of dimension. 784 is the dimension of MNIST data.
    y_target = tf.placeholder("float", shape=[None, 10], name = 'y_target') # shape argument is optional, but this is useful to debug.


    # all the variables are allocated in GPU memory
    W1 = tf.Variable(tf.zeros([784, 256]), name = 'W1')   # create (784 * 256) matrix
    b1 = tf.Variable(tf.zeros([256]), name = 'b1')        # create (1 * 256) vector
    h1 = tf.sigmoid(tf.matmul(x, W1) + b1, name = 'h1')   # compute --> sigmoid(weighted summation)

    # Repeat again
    W2 = tf.Variable(tf.zeros([256, 10]), name = 'W2')     # create (256 * 10) matrix
    b2 = tf.Variable(tf.zeros([10]), name = 'b2')          # create (1 * 10) vector
    y = tf.nn.softmax(tf.matmul(h1, W2) + b2, name = 'y')  # compute classification --> softmax(weighted summation)


    # define the Loss function
    cross_entropy = -tf.reduce_sum(y_target*tf.log(y), name = 'cross_entropy')


    # define optimization algorithm
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)



    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))
    # correct_prediction is list of boolean which is the result of comparing(model prediction , data)


    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
    # tf.cast() : changes true -> 1 / false -> 0
    # tf.reduce_mean() : calculate the mean

    # Create Session
    sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) # open a session which is a envrionment of computation graph.
    sess.run(tf.global_variables_initializer())# initialize the variables

    
    # create summary of parameters
    tf.summary.histogram('weights_1', W1)
    tf.summary.histogram('weights_2', W2)
    tf.summary.histogram('y', y)
    tf.summary.scalar('cross_entropy', cross_entropy)
    merged = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter("/tmp/mlp", sess.graph)

    
    # training the MLP
    for i in range(5001): # minibatch iteraction
        batch = mnist.train.next_batch(100) # minibatch size
        sess.run([train_step], feed_dict={x: batch[0], y_target: batch[1]}) # placeholder's none length is replaced by i:i+100 indexes

        if i%500 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_target: batch[1]})
            print ("step %d, training accuracy: %.3f"%(i, train_accuracy))



            # calculate the summary and write.
            summary = sess.run(merged, feed_dict={x:batch[0], y_target: batch[1]})
            summary_writer.add_summary(summary , i)

    # for given x, y_target data set
    print  ("validation accuracy: %g"% sess.run(accuracy, feed_dict={x: mnist.validation.images, y_target: mnist.validation.labels}))
    sess.close()

MLP()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy: 0.120
step 500, training accuracy: 0.560
step 1000, training accuracy: 0.860
step 1500, training accuracy: 0.920
step 2000, training accuracy: 0.970
step 2500, training accuracy: 0.930
step 3000, training accuracy: 0.940
step 3500, training accuracy: 0.960
step 4000, training accuracy: 0.940
step 4500, training accuracy: 0.990
step 5000, training accuracy: 0.980
validation accuracy: 0.9416


### TensorBoard 실행하기
위와 같이 코드를 수정했다면, 이제 리눅스 shell로 이동한 후, tensorboard를 실행시킨다.
혹은 IPython에서 new -> terminal을 클릭하여 아래의 명령을 실행할 수도 있다. 종료는 ctrl + x를 입력한다.


tensorboard --logdir=/tmp/mlp --port=6006


(만약 위 명령어 실행시 문제가 생기는 경우 다음을 실행)
<br>cd tensorflow/tensorflow/tensorboard
<br>python tensorboard.py --logdir=/tmp/mlp --port=6006
<br>https://github.com/tensorflow/tensorflow/blob/r0.10/tensorflow/tensorboard/README.md



그다음 새로운 웹브라우저를 열어 IP:6006/#graphs 에 접속하면 아래와 같은 그림을 볼 수 있다.
<br>(ex) http://192.168.99.100:6006/#graphs



![](images/mlp_total.png)

이번에는 이미지 인식 분야에서 가장 성공적으로 쓰이고 있는 Convolutional Neural Networks를 실습해본다.

### Convolutional Neural Networks
Convolutional Neural Networks, CNN은 아래와 같은 Convolutional Layer를 여러층 가진 딥러닝 모델을 뜻한다.




![](images/Cnn_layer.png)
(출처: http://ufldl.stanford.edu/tutorial/images/Cnn_layer.png)

최근에는 아래와 같이 매우 깊은 층으로 Convolutional Layer를 쌓고 매우 방대한 량의 이미지를 학습하는 경우가 많다.

![](images/big_cnn.png)
(출처: https://qph.is.quoracdn.net/main-qimg-cf89aa517e5b641dc8e41e7a57bafc2c?convert_to_webp=true)

이번에는 간단한 구조를 가진 CNN을 구현하고 방금전에 사용했던 MNIST 데이터를 학습시켜 보고, MLP와의 성능 차이를 비교해본다.

아래의 코드를 보면 MLP와 전체 구조는 매우 유사한데, 중간에 conv2d나 max-pool을 비롯해 처음 보는 여러 연산들이 추가 된 것을 알 수 있다. 또한 CNN을 효과적으로 학습하기 위해서는 Weight의 초기화를 0으로 하는 것이 아니라, 랜덤으로 해주어야하는데, 여기서는 가우시안을 이용하여 초기화 하였다. 그밖에 dropout과 relu, Adam 등이 추가로 사용되었다.

각 함수와 연산들의 자세한 설명은 아래 코드를 보면서 하나하나 분석해보자.

In [42]:
%matplotlib inline
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.reset_default_graph() # remove the previous computation graph

def CNN():
    # download the mnist data.
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True) 


    # placeholder is used for feeding data.
    x = tf.placeholder("float", shape=[None, 784], name = 'x') # none represents variable length of dimension. 784 is the dimension of MNIST data.
    y_target = tf.placeholder("float", shape=[None, 10], name = 'y_target') # shape argument is optional, but this is useful to debug.


    
    # reshape input data
    x_image = tf.reshape(x, [-1,28,28,1], name="x_image")
    
    # Build a convolutional layer and maxpooling with random initialization
    W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1), name="W_conv1") # W is [row, col, channel, feature]
    b_conv1 = tf.Variable(tf.zeros([32]), name="b_conv1")
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1, name="h_conv1")
    h_pool1 = tf.nn.max_pool( h_conv1 , ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name = "h_pool1")
    
    # Repeat again with 64 number of filters
    W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1), name="W_conv2") # W is [row, col, channel, feature]
    b_conv2 = tf.Variable(tf.zeros([64]), name="b_conv2")
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2, name="h_conv2")
    h_pool2 = tf.nn.max_pool( h_conv2 , ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name = "h_pool2")
    
    # Build a fully connected layer
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64], name="h_pool2_flat")
    W_fc1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1), name = 'W_fc1')
    b_fc1 = tf.Variable(tf.zeros([1024]), name = 'b_fc1')
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1, name="h_fc1")

    
    # Dropout Layer
    keep_prob = tf.placeholder("float", name="keep_prob")
    h_fc1 = tf.nn.dropout(h_fc1, keep_prob, name="h_fc1_drop")
    
    # Build a fully connected layer with softmax 
    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1), name = 'W_fc2')
    b_fc2 = tf.Variable(tf.zeros([10]), name = 'b_fc2')
    y=tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2, name="y")
    



    # define the Loss function
    cross_entropy = -tf.reduce_sum(y_target*tf.log(y), name = 'cross_entropy')
    

    # define optimization algorithm
    #train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)



    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))
    # correct_prediction is list of boolean which is the result of comparing(model prediction , data)


    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
    # tf.cast() : changes true -> 1 / false -> 0
    # tf.reduce_mean() : calculate the mean

    # Create Session
    sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))   # open a session which is a envrionment of computation graph.
    sess.run(tf.global_variables_initializer())# initialize the variables
    
    

    # training the MLP
    for i in range(5001): # minibatch iteraction
        batch = mnist.train.next_batch(100) # minibatch size
        sess.run(train_step, feed_dict={x: batch[0], y_target: batch[1], keep_prob: 0.5}) # placeholder's none length is replaced by i:i+100 indexes

        if i%500 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_target: batch[1], keep_prob: 1})
            print ("step %d, training accuracy: %.3f"%(i, train_accuracy))

    # for given x, y_target data set
    print  ("validation accuracy: %g"% sess.run(accuracy, feed_dict={x: mnist.validation.images[0:150], y_target: mnist.validation.labels[0:150], keep_prob: 1}))
    sess.close()
CNN()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy: 0.040
step 500, training accuracy: 0.990
step 1000, training accuracy: 1.000
step 1500, training accuracy: 0.980
step 2000, training accuracy: 0.970
step 2500, training accuracy: 0.980
step 3000, training accuracy: 1.000
step 3500, training accuracy: 0.970
step 4000, training accuracy: 1.000
step 4500, training accuracy: 1.000
step 5000, training accuracy: 1.000
validation accuracy: 0.98


학습 결과, MLP보다 성능이 훨씬 높아진 것을 알 수 있고 동시에 학습 속도가 보다 느려진 것을 확인할 수 있다. (GPU 메모리가 충분하지 않아, test 정확도는 150개의 이미지에 대해서만 측정하였다.)

이제 마찬가지로 TensorBoard를 통해서 우리가 만든 Computation Graph를 직접 눈으로 확인해보면 다음과 같이 그려진다.

![](images/cnn.png)

### Regularization: Early Stopping 및 Index Shuffling 
그런데 실제로 Deep Learning을 학습할 때에는 우리가 정한 횟수만큼의 iteration을 무조건 반복하는 것이 아니라, 적당히 학습이 완료되었다고 생각되면 학습을 중단하는 Early Stopping을 해야한다. 이것을 하지 않고 무조건 정해진 iteration을 하게되면, 모델이 주어진 데이터에만 과도하게 학습하여, 보지않은 데이터에 대한 일반화 성능이 떨어지는 overfitting이 일어나게 된다. 따라서 Early Stopping을 통해 이러한 일이 일어나기 전에 학습을 중단해야 한다.

또한 위의 MNIST 데이터는 이미 구현된 함수를 통해 미리 순서가 뒤섞이고, one-hot coding이 된 데이터를 필요한 개수만큼 가져올 수 있었다. 그러나 실제로 자신의 데이터를 학습시키기 위해서는 이러한 과정도 파이썬의 numpy를 이용해 구현해 주어야한다.

그래서 이번 예제에서는 아주 간단하면서 유명한 Iris 데이터를 이용해 위의 구현을 실습해보도록 한다.

Iris data : 50개*3종의 iris 꽃에서 4종류의 feature를 추출한 데이터
<br>https://en.wikipedia.org/wiki/Iris_flower_data_set

In [43]:
%matplotlib inline
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.base import load_iris
import numpy as np
tf.reset_default_graph()

def MLP_iris():
    # load the iris data.
    iris = load_iris()

    np.random.seed(0)
    random_index = np.random.permutation(150)

    iris_data = iris.data[random_index]
    iris_target = iris.target[random_index]
    iris_target_onehot = np.zeros((150, 3))
    iris_target_onehot[np.arange(150), iris_target] = 1

    accuracy_list = []
    
    # build computation graph
    x = tf.placeholder("float", shape=[None, 4], name = 'x')
    y_target = tf.placeholder("float", shape=[None, 3], name = 'y_target')

    W1 = tf.Variable(tf.zeros([4, 128]), name = 'W1')
    b1 = tf.Variable(tf.zeros([128]), name = 'b1')
    h1 = tf.sigmoid(tf.matmul(x, W1) + b1, name = 'h1')

    W2 = tf.Variable(tf.zeros([128, 3]), name = 'W2')
    b2 = tf.Variable(tf.zeros([3]), name = 'b2')
    y = tf.nn.softmax(tf.matmul(h1, W2) + b2, name = 'y')

    cross_entropy = -tf.reduce_sum(y_target*tf.log(y), name = 'cross_entropy')

    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))

    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 

    sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
    sess.run(tf.global_variables_initializer())

    for i in range(500):
        sess.run(train_step, feed_dict={x: iris_data[0:100], y_target: iris_target_onehot[0:100]})

        train_accuracy = sess.run(accuracy, feed_dict={x: iris_data[0:100], y_target: iris_target_onehot[0:100]})
        validation_accuracy = sess.run(accuracy, feed_dict={x: iris_data[100:], y_target: iris_target_onehot[100:]})
        if i%5 is 0:
            print ("step %d, training accuracy: %.3f / validation accuracy: %.3f" %(i, train_accuracy, validation_accuracy))

        accuracy_list.append(validation_accuracy)
        
        if i >= 50:
            if validation_accuracy - np.mean(accuracy_list[len(accuracy_list)/2:]) <= 0.01 :
                break
            
    sess.close()

MLP_iris()

step 0, training accuracy: 0.360 / validation accuracy: 0.280
step 5, training accuracy: 0.350 / validation accuracy: 0.180
step 10, training accuracy: 0.040 / validation accuracy: 0.020
step 15, training accuracy: 0.470 / validation accuracy: 0.500
step 20, training accuracy: 0.660 / validation accuracy: 0.660
step 25, training accuracy: 0.660 / validation accuracy: 0.660
step 30, training accuracy: 0.660 / validation accuracy: 0.660
step 35, training accuracy: 0.660 / validation accuracy: 0.660
step 40, training accuracy: 0.700 / validation accuracy: 0.680
step 45, training accuracy: 0.730 / validation accuracy: 0.760
step 50, training accuracy: 0.780 / validation accuracy: 0.800
step 55, training accuracy: 0.790 / validation accuracy: 0.800
step 60, training accuracy: 0.810 / validation accuracy: 0.820
step 65, training accuracy: 0.810 / validation accuracy: 0.820
step 70, training accuracy: 0.830 / validation accuracy: 0.840
step 75, training accuracy: 0.860 / validation accuracy: 

### Save & Load Parameters
그러면 이번에는 이렇게 학습한 모델을 필요한 시점에서 저장하고, 다시 불러오는 기능을 구현해보자. 실제 문제를 풀 때에는 위와 같이 빠른 학습이 불가능하기 때문에 반드시 학습된 모델을 저장하고, 필요할 때 불러오는 기능이 필요하다.

In [45]:
%matplotlib inline
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()

def MLP():
    # download the mnist data.
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True) 


    # placeholder is used for feeding data.
    x = tf.placeholder("float", shape=[None, 784], name = 'x') # none represents variable length of dimension. 784 is the dimension of MNIST data.
    y_target = tf.placeholder("float", shape=[None, 10], name = 'y_target') # shape argument is optional, but this is useful to debug.


    # all the variables are allocated in GPU memory
    W1 = tf.Variable(tf.zeros([784, 256]), name = 'W1')   # create (784 * 256) matrix
    b1 = tf.Variable(tf.zeros([256]), name = 'b1')        # create (1 * 256) vector
    h1 = tf.sigmoid(tf.matmul(x, W1) + b1, name = 'h1')   # compute --> sigmoid(weighted summation)

    # Repeat again
    W2 = tf.Variable(tf.zeros([256, 10]), name = 'W2')     # create (256 * 10) matrix
    b2 = tf.Variable(tf.zeros([10]), name = 'b2')          # create (1 * 10) vector
    y = tf.nn.softmax(tf.matmul(h1, W2) + b2, name = 'y')  # compute classification --> softmax(weighted summation)


    # define the Loss function
    cross_entropy = -tf.reduce_sum(y_target*tf.log(y), name = 'cross_entropy')


    # define optimization algorithm
    train_step = tf.train.GradientDescentOptimizer(0.01, name='GradientDescent').minimize(cross_entropy)



    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_target, 1))
    # correct_prediction is list of boolean which is the result of comparing(model prediction , data)


    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"), name='accuracy') 
    # tf.cast() : changes true -> 1 / false -> 0
    # tf.reduce_mean() : calculate the mean

    # Create Session
    sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) # open a session which is a envrionment of computation graph.
    sess.run(tf.global_variables_initializer())# initialize the variables

    # Create Directory
    import os
    if not os.path.exists('Checkpoint'):
        os.makedirs('Checkpoint')
        
    # Create Saver
    saver = tf.train.Saver(max_to_keep=11)
    #saver.restore(sess, os.path.join('Checkpoint', "mlp-5000"))
    
    # training the MLP
    for i in range(5001): # minibatch iteraction
        batch = mnist.train.next_batch(100) # minibatch size
        sess.run(train_step, feed_dict={x: batch[0], y_target: batch[1]}) # placeholder's none length is replaced by i:i+100 indexes

        if i%500 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_target: batch[1]})
            print ("step %d, training accuracy: %.3f"%(i, train_accuracy))

            saver.save(sess,  os.path.join('Checkpoint', "mlp") , global_step=i)
            

    # for given x, y_target data set
    print  ("validation accuracy: %g"% sess.run(accuracy, feed_dict={x: mnist.validation.images[0:150], y_target: mnist.validation.labels[0:150]}))
    sess.close()

MLP()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy: 0.980
step 500, training accuracy: 0.990
step 1000, training accuracy: 0.910
step 1500, training accuracy: 0.980
step 2000, training accuracy: 0.970
step 2500, training accuracy: 0.970
step 3000, training accuracy: 0.970
step 3500, training accuracy: 0.990
step 4000, training accuracy: 0.990
step 4500, training accuracy: 0.970
step 5000, training accuracy: 0.950
validation accuracy: 0.96


### Load Pre-defined Computation Graph and Trained Parameters
위의 예제에서는 한가지 부족한 점이 있다. 그것은 바로 학습된 parameter를 저장하고 불러올 수는 있지만, 이미 만들어진 Computation Graph를 불러오는 기능이 빠져있다. 이번에는 Computation Graph를 불러오는 부분까지 포함해서 실습해보자.

In [46]:
%matplotlib inline
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.reset_default_graph()


sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) # open a session which is a envrionment of computation graph.
sess.run(tf.global_variables_initializer())# initialize the variables

new_saver = tf.train.import_meta_graph('Checkpoint/mlp-0.meta')
new_saver.restore(sess, 'Checkpoint/mlp-500')

train_step = sess.graph.get_operation_by_name('GradientDescent')
x = sess.graph.get_tensor_by_name('x:0')
y_target = sess.graph.get_tensor_by_name('y_target:0')
accuracy = sess.graph.get_tensor_by_name('accuracy:0')

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# training the MLP
for i in range(5001): # minibatch iteraction
    batch = mnist.train.next_batch(100) # minibatch size
    sess.run(train_step, feed_dict={x: batch[0], y_target: batch[1]}) # placeholder's none length is replaced by i:i+100 indexes

    if i%500 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x:batch[0], y_target: batch[1]})
        print ("step %d, training accuracy: %.3f"%(i, train_accuracy))

# for given x, y_target data set
print  ("validation accuracy: %g"% sess.run(accuracy, feed_dict={x: mnist.validation.images, y_target: mnist.validation.labels}))
sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy: 0.980
step 500, training accuracy: 0.990
step 1000, training accuracy: 0.960
step 1500, training accuracy: 0.990
step 2000, training accuracy: 0.960
step 2500, training accuracy: 0.990
step 3000, training accuracy: 0.990
step 3500, training accuracy: 0.970
step 4000, training accuracy: 0.970
step 4500, training accuracy: 0.950
step 5000, training accuracy: 1.000
validation accuracy: 0.9544


### Recurrent Neural Networks
Recurrent Neural Networks, 이하 RNN는 다음과 같은 구조를 가진 모델이다. RNN은 히든레이어에서 자기자신을 향하는 weight를 이용해 데이터간의 시간관계를 학습할 수 있다. 이러한 문제들을 시계열 학습이라고 부르며, 기존에 널리 쓰이던 Hidden Markov Model을 뉴럴넷을 이용해 구현했다고 볼 수 있다.


![](images/rnn.png)


![](images/unroll_rnn.png)

위의 구조는 1개의 Recurrent weight를 가진 hidden node이다. 이러한 hidden node들이 여러개를 모여 1개의 RNN layer를 형성하고, 이것이 다시 deep 하게 쌓이는 모델 또한 가능하다.(그러나 RNN은 deep 하게 쌓을 경우 학습이 쉽지 않다.)

RNN의 경우 MLP나 CNN에 비해서 구현이 다소 복잡하다. 따라서 RNN은 TensorFlow에서 제공하는 추상화된 API 들을 이용해서 구현하는 것이 일반적이다.

![](images/longterm.png)
(출처: http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

- tf.contrib.rnn.BasicRNNCell : 기본 RNN cell이외에 매우 다양한 cell을 사용할 수 있다.
https://www.tensorflow.org/api_guides/python/contrib.rnn#Core_RNN_Cells_for_use_with_TensorFlow_s_core_RNN_methods

- tf.contrib.seq2seq.sequence_loss : N개의 sequence의 cross-entropy를 weighted sum 한 Loss. weight는 0or1의 masking용도로써 쓰인다.
https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/sequence_loss

### Simple toy seq2seq with RNN / LSTM
<img src="images/rnn_seq2seq.jpg">
(이미지 출처: http://smile2x.tistory.com/archive/201605)
(소스코드 출처: https://github.com/hunkim/DeepLearningZeroToAll)

In [57]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(777)  # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0 
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello
y_one_hot = [[[0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 0, 1]]]  # o 4

input_dim = 5  # one-hot size
hidden_size = 10  # output from the LSTM. 5 to directly predict one-hot
output_dim = 5
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6

X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])  # X one-hot
Y = tf.placeholder(tf.float32, [None, sequence_length, output_dim])  # Y one-hot

cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
#cell = tf.contrib.rnn.GRUCell(num_units=hidden_size)
#cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)
print "outputs: ",outputs

outputs_flat = tf.reshape(outputs, [-1 , hidden_size])
print "outputs_flat: ",outputs_flat
w = tf.Variable(tf.random_normal([hidden_size, output_dim]))
b = tf.Variable(tf.random_normal([output_dim]))
y_prediction = tf.nn.softmax(tf.matmul(outputs_flat, w) + b)
print "y_prediction:", y_prediction
y_prediction = tf.reshape(y_prediction, [-1 , sequence_length, output_dim])
print "y_prediction:", y_prediction
print Y

y_label_pred = tf.argmax(y_prediction, 2)
    
loss = -tf.reduce_sum(Y*tf.log(y_prediction), name = 'cross_entropy')

train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_one_hot})
        if i %100 ==0:
            _y_prediction, _y_label_pred = sess.run([y_prediction, y_label_pred], feed_dict={X: x_one_hot})
            #print i, "loss:", l, "_y_prediction: ", _y_prediction, "prediction: ", _y_label_pred, "true Y: ", y_data
            print i, "loss:", l, "prediction: ", _y_label_pred, "true Y: ", y_data

            # print char using dic
            #result_str = [idx2char[c] for c in np.squeeze(_prediction)]
            #print "Prediction str: ", ''.join(result_str)

outputs:  Tensor("rnn/transpose:0", shape=(1, 6, 10), dtype=float32)
outputs_flat:  Tensor("Reshape:0", shape=(6, 10), dtype=float32)
y_prediction: Tensor("Softmax:0", shape=(6, 5), dtype=float32)
y_prediction: Tensor("Reshape_1:0", shape=(1, 6, 5), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 6, 5), dtype=float32)
0 loss: 13.3657 prediction:  [[1 0 1 1 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
100 loss: 0.000930944 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
200 loss: 0.000502015 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
300 loss: 0.000329147 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
400 loss: 0.000234132 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
500 loss: 0.000175122 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
600 loss: 0.00013602 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
700 loss: 0.000108482 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
800 loss: 8.88118e-05 prediction:  [

### Web log Sequence Classification

- tf.one_hot : 자동으로 one_hot encoding을 해주는 함수
<br>https://www.tensorflow.org/api_docs/python/tf/one_hot


In [58]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
tf.reset_default_graph()

train_x = np.loadtxt('1000_index_x.txt')
train_y = np.loadtxt('1000_index_y.txt')

nclasses = 52739 #len(items)
nsteps = 4

dimhidden = 100
dimoutput = nclasses

x = tf.placeholder("int32", [None, nsteps])
y = tf.placeholder("int32", [None])
x_one_hot = tf.one_hot(x, nclasses, on_value=1.0, off_value=0.0 , dtype='float')
y_one_hot = tf.one_hot(y, nclasses, on_value=1.0, off_value=0.0, dtype='float')

cell = tf.contrib.rnn.BasicLSTMCell(dimhidden)
outputs, states = tf.nn.dynamic_rnn(cell, x_one_hot,  dtype=tf.float32)
last_output = outputs[:, -1, :]

w = tf.Variable(tf.random_normal([dimhidden, dimoutput]))
b = tf.Variable(tf.random_normal([dimoutput]))
prediction = tf.nn.softmax(tf.matmul(last_output, w) + b)

cost = -tf.reduce_mean( y_one_hot*tf.log(prediction+0.00000001))
optm     = tf.train.AdamOptimizer().minimize(cost) 
accr     = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(prediction,1), tf.argmax(y_one_hot,1)), tf.float32))

sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
sess.run(tf.global_variables_initializer())

epochs = 1000
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

for i in range(epochs):
    _, _cost, _accr = sess.run([optm, cost, accr], feed_dict={x: x_train, y: y_train})
    if i%10 == 0 :
        print i, " : ", _cost , _accr
    
    if i%50 == 0 :
        _, _cost, test_acc = sess.run([optm, cost, accr], feed_dict={x: x_test, y: y_test})
        print "Test accuracy: %.3f" % (test_acc)


0  :  0.000216435 0.0
Test accuracy: 0.000
10  :  0.000215641 0.0
20  :  0.0002145 0.0
30  :  0.000213176 0.0
40  :  0.000211726 0.0
50  :  0.000210169 0.0
Test accuracy: 0.000
60  :  0.000208456 0.0
70  :  0.000206732 0.0
80  :  0.000204878 0.0
90  :  0.000202902 0.0
100  :  0.000200793 0.0
Test accuracy: 0.000
110  :  0.000198449 0.00125
120  :  0.000196054 0.00375
130  :  0.000193422 0.00375
140  :  0.000190552 0.005
150  :  0.000187414 0.00625
Test accuracy: 0.000
160  :  0.000183845 0.01375
170  :  0.00018011 0.01875
180  :  0.000175908 0.02875
190  :  0.000171191 0.0375
200  :  0.000165855 0.05625
Test accuracy: 0.000
210  :  0.000159536 0.08375
220  :  0.000152541 0.125
230  :  0.00014414 0.17
240  :  0.000133858 0.24375
250  :  0.000120906 0.33375
Test accuracy: 0.015
260  :  0.000103929 0.46625
270  :  8.31161e-05 0.5775
280  :  5.80032e-05 0.71875
290  :  3.29999e-05 0.8325
300  :  1.52336e-05 0.9275
Test accuracy: 0.030
310  :  6.95845e-06 0.97625
320  :  2.93543e-06 0.9925


KeyboardInterrupt: 

## * Open-source TensorFlow Implementation

아래 링크는 TensorFlow로 구현되어 공개된 여러 오픈소스 프로젝트들을 모아서 정리해 둔 페이지들이다. 이중 본인의 연구 분야와 관련있는 프로젝트를 clone, 수정하여 사용할 경우 개발시간을 크게 단축할 수 있다.

https://github.com/tensorflow/models : Syntax Net, Magenta, Image2Txt
<br>https://github.com/TensorFlowKR/awesome_tensorflow_implementations
<br>https://github.com/aikorea/awesome-rl

유명한 오픈소스 몇가지를 살펴보자.